In [1]:
import os
import pandas as pd

from llm_political_analysis.modules.summarize import summarize_file

In [2]:
issue_list = ['european_union', 'taxation', 'lifestyle', 'immigration', 'environment', 'decentralization']

In [3]:
calibration_list = os.listdir('../data/plaintext/new_calibration')

In [4]:
calibration_list

['Belgium - BEL 2014 - BEL 2014  New Flemish Alliance N VA.txt',
 'Austria - AU 2019 - AU 2019 SDP.txt',
 'Czech - CZ 2010 - CZ 2010 Civ Dem ODS.txt',
 'Poland - POL 2019 Civic Coalition PO.txt',
 'Denmark - DEN 2019 SPP SF.txt',
 'Sweden - SWE 2010 - SWE 2010 Moderate.txt',
 'Netherlands - NL 2010 - NL 2010 Green GL.txt',
 'Hungary - HUN 2014 - HUN 2010 Mvmnt for better Hungary JOBBIK.txt']

In [5]:
calibration_list_stem = [x.split(".")[0] for x in calibration_list]  

In [6]:
calibration_list_stem

['Belgium - BEL 2014 - BEL 2014  New Flemish Alliance N VA',
 'Austria - AU 2019 - AU 2019 SDP',
 'Czech - CZ 2010 - CZ 2010 Civ Dem ODS',
 'Poland - POL 2019 Civic Coalition PO',
 'Denmark - DEN 2019 SPP SF',
 'Sweden - SWE 2010 - SWE 2010 Moderate',
 'Netherlands - NL 2010 - NL 2010 Green GL',
 'Hungary - HUN 2014 - HUN 2010 Mvmnt for better Hungary JOBBIK']

In [7]:
summary_list = os.listdir('../data/summaries')

In [8]:
summary_list

['1 Test - CRO 2003 Democratic Union_summary.txt',
 '0 Calibration - GER 2002 CDU-CSU_summary.txt',
 '0 Calibration - AU 99 FPO_summary.txt',
 '0 Calibration - CZ 2006 Green_summary.txt',
 '1 Test - BUL 2014 Attack_summary.txt',
 '0 Calibration - NL 2003 VVD txt_summary.txt',
 '0 Calibration - HUN 2010 Soc Dem_summary.txt',
 '.gitkeep',
 'Belgium - BEL 2014 - BEL 2014  New Flemish Alliance N VA_summary.txt',
 '0 Calibration - SLE 2014 United Left_summary.txt',
 '0 Calibration - IRL 1989 Lab txt_summary.txt',
 '1 Test - FR 2002 FN txt_summary.txt',
 '1 Test - SWE 2010 Moderate_summary.txt',
 '0 Calibration - UK 2019 Lib Dem_summary.txt',
 '1 Test - POL 2019 Civic Coalition_summary.txt',
 '1 Test - POR1999 Soc Dem_summary.txt',
 '1 Test - NOR 1989 Conservative txt_summary.txt',
 '1 Test - SLO 2006 Xtian Dem Movement_summary.txt']

In [11]:
for cal_file in calibration_list_stem:
    if cal_file + "_summary.txt" not in summary_list:
        print(f'Summarizing {cal_file}')
        summarize_file('../data/plaintext/new_calibration/' + cal_file + ".txt", issue_list)

Summarizing Austria - AU 2019 - AU 2019 SDP
Waiting for 47 seconds to avoid token limit
Waiting for 45 seconds to avoid token limit
Summarized so far: 3 out of 3 chunks
Combining summaries into one final summary
Final summary length: 5325 characters 

Summarizing Czech - CZ 2010 - CZ 2010 Civ Dem ODS
Waiting for 43 seconds to avoid token limit
Summarized so far: 2 out of 2 chunks
Combining summaries into one final summary
Final summary length: 5374 characters 

Summarizing Poland - POL 2019 Civic Coalition PO
Waiting for 47 seconds to avoid token limit
Summarized so far: 2 out of 2 chunks
Combining summaries into one final summary
Final summary length: 5276 characters 

Summarizing Denmark - DEN 2019 SPP SF
Summarized so far: 1 out of 1 chunks
Final summary length: 4284 characters 

Summarizing Sweden - SWE 2010 - SWE 2010 Moderate
Waiting for 45 seconds to avoid token limit
Summarized so far: 2 out of 2 chunks
Combining summaries into one final summary
Final summary length: 4286 chara

In [23]:
ches_results = pd.read_csv('../data/ches_scores/CHES cal-test-data.csv')


In [24]:
ches_results

,year,country,party,file,EU,TaxSpend,Social_lifestyle,Immigration,Decentralization,Environment,Cal_Test
0,2019,AUS,SPO,Austria - AU 2019 - AU 2019 SDP,6.1,2.5,2.4,4.7,6.0,4.9,C
1,2014,BEL,N-VA,Belgium - BEL 2014 - BEL 2014 New Flemish All...,5.0,9.0,3.6,7.6,2.0,7.6,C
2,2010,CZE,ODS,Czech - CZ 2010 - CZ 2010 Civ Dem ODS,3.0,8.2,5.4,6.5,6.3,8.1,C
3,2019,DK,SF,Denmark - DEN 2019 SPP SF,5.1,1.8,1.6,4.3,5.1,3.4,C
4,2014,HUN,JOBBIK,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...,1.2,4.3,9.8,9.3,5.1,5.8,C
5,2010,NETH,GL,Netherlands - NL 2010 - NL 2010 Green GL,6.2,2.4,1.4,2.6,4.5,1.5,C
6,2019,POL,PO,Poland - POL 2019 Civic Coalition PO,6.7,5.3,4.2,4.6,2.8,5.1,C
7,2010,SWE,M,Sweden - SWE 2010 - SWE 2010 Moderate,6.3,7.6,3.2,5.4,5.2,5.9,C
8,2006,AUS,Freedom Party of Austria,Austria - AU 2006 - AU 2006 FPO,1.8,5.7,9.5,9.8,6.2,NaN,T
9,2010,BEL,ECOLO,Belgium - BEL 2010 - BEL 2010 Green Ecolo,6.0,1.5,1.2,1.0,5.8,1.1,T


In [25]:
ches_results['Calibration File'] = ches_results['file'].apply(lambda x: x+'_summary.txt')
ches_results.loc[ches_results['Cal_Test'] == 'T', 'Calibration File'] = None

In [26]:
ches_results['Calibration File'].value_counts()

Calibration File
Austria - AU 2019 - AU 2019 SDP_summary.txt                                  1
Belgium - BEL 2014 - BEL 2014  New Flemish Alliance N VA_summary.txt         1
Czech - CZ 2010 - CZ 2010 Civ Dem ODS_summary.txt                            1
Denmark - DEN 2019 SPP SF_summary.txt                                        1
Hungary - HUN 2014 - HUN 2010 Mvmnt for better Hungary JOBBIK_summary.txt    1
Netherlands - NL 2010 - NL 2010 Green GL_summary.txt                         1
Poland - POL 2019 Civic Coalition PO_summary.txt                             1
Sweden - SWE 2010 - SWE 2010 Moderate_summary.txt                            1
Name: count, dtype: int64

In [27]:
ches_results

,year,country,party,file,EU,TaxSpend,Social_lifestyle,Immigration,Decentralization,Environment,Cal_Test,Calibration File
0,2019,AUS,SPO,Austria - AU 2019 - AU 2019 SDP,6.1,2.5,2.4,4.7,6.0,4.9,C,Austria - AU 2019 - AU 2019 SDP_summary.txt
1,2014,BEL,N-VA,Belgium - BEL 2014 - BEL 2014 New Flemish All...,5.0,9.0,3.6,7.6,2.0,7.6,C,Belgium - BEL 2014 - BEL 2014 New Flemish All...
2,2010,CZE,ODS,Czech - CZ 2010 - CZ 2010 Civ Dem ODS,3.0,8.2,5.4,6.5,6.3,8.1,C,Czech - CZ 2010 - CZ 2010 Civ Dem ODS_summary.txt
3,2019,DK,SF,Denmark - DEN 2019 SPP SF,5.1,1.8,1.6,4.3,5.1,3.4,C,Denmark - DEN 2019 SPP SF_summary.txt
4,2014,HUN,JOBBIK,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...,1.2,4.3,9.8,9.3,5.1,5.8,C,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...
5,2010,NETH,GL,Netherlands - NL 2010 - NL 2010 Green GL,6.2,2.4,1.4,2.6,4.5,1.5,C,Netherlands - NL 2010 - NL 2010 Green GL_summa...
6,2019,POL,PO,Poland - POL 2019 Civic Coalition PO,6.7,5.3,4.2,4.6,2.8,5.1,C,Poland - POL 2019 Civic Coalition PO_summary.txt
7,2010,SWE,M,Sweden - SWE 2010 - SWE 2010 Moderate,6.3,7.6,3.2,5.4,5.2,5.9,C,Sweden - SWE 2010 - SWE 2010 Moderate_summary.txt
8,2006,AUS,Freedom Party of Austria,Austria - AU 2006 - AU 2006 FPO,1.8,5.7,9.5,9.8,6.2,NaN,T,None
9,2010,BEL,ECOLO,Belgium - BEL 2010 - BEL 2010 Green Ecolo,6.0,1.5,1.2,1.0,5.8,1.1,T,None


In [37]:
ches_results_simple = ches_results[['Calibration File', 'TaxSpend', 'Social_lifestyle', 'Immigration', 'Decentralization','Environment', 'EU']]
ches_results_simple = ches_results_simple[ches_results_simple['Calibration File'].notnull()]

In [42]:
ches_results_long = ches_results_simple.melt(id_vars='Calibration File', var_name='Dimension', value_name='Expert mean')[['Dimension','Calibration File','Expert mean']]

In [46]:
issue_map = {'EU': 'european_union',
 'TaxSpend': 'taxation',
 'Social_lifestyle': 'lifestyle',
 'Immigration': 'immigration',
 'Environment': 'environment',
 'Decentralization': 'decentralization'}

In [47]:
ches_results_long['issue'] = ches_results_long['Dimension'].apply(lambda x: issue_map[x])

In [48]:
ches_results_long

,Dimension,Calibration File,Expert mean,issue
0,TaxSpend,Austria - AU 2019 - AU 2019 SDP_summary.txt,2.5,taxation
1,TaxSpend,Belgium - BEL 2014 - BEL 2014 New Flemish All...,9.0,taxation
2,TaxSpend,Czech - CZ 2010 - CZ 2010 Civ Dem ODS_summary.txt,8.2,taxation
3,TaxSpend,Denmark - DEN 2019 SPP SF_summary.txt,1.8,taxation
4,TaxSpend,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...,4.3,taxation
5,TaxSpend,Netherlands - NL 2010 - NL 2010 Green GL_summa...,2.4,taxation
6,TaxSpend,Poland - POL 2019 Civic Coalition PO_summary.txt,5.3,taxation
7,TaxSpend,Sweden - SWE 2010 - SWE 2010 Moderate_summary.txt,7.6,taxation
8,Social_lifestyle,Austria - AU 2019 - AU 2019 SDP_summary.txt,2.4,lifestyle
9,Social_lifestyle,Belgium - BEL 2014 - BEL 2014 New Flemish All...,3.6,lifestyle


In [49]:
final_prompt_setup = ches_results_long[['issue','Calibration File','Expert mean']].where(ches_results_long['Calibration File'].notna()).dropna()

In [50]:
final_prompt_setup['Expert mean'] = pd.to_numeric(final_prompt_setup['Expert mean'], errors='coerce').round(0).apply(lambda x: str(int(x)) if not pd.isna(x) else 'NA')

In [51]:
final_prompt_setup

,issue,Calibration File,Expert mean
0,taxation,Austria - AU 2019 - AU 2019 SDP_summary.txt,2
1,taxation,Belgium - BEL 2014 - BEL 2014 New Flemish All...,9
2,taxation,Czech - CZ 2010 - CZ 2010 Civ Dem ODS_summary.txt,8
3,taxation,Denmark - DEN 2019 SPP SF_summary.txt,2
4,taxation,Hungary - HUN 2014 - HUN 2010 Mvmnt for better...,4
5,taxation,Netherlands - NL 2010 - NL 2010 Green GL_summa...,2
6,taxation,Poland - POL 2019 Civic Coalition PO_summary.txt,5
7,taxation,Sweden - SWE 2010 - SWE 2010 Moderate_summary.txt,8
8,lifestyle,Austria - AU 2019 - AU 2019 SDP_summary.txt,2
9,lifestyle,Belgium - BEL 2014 - BEL 2014 New Flemish All...,4


In [52]:
final_prompt_setup.to_csv('../data/ches_scores/final_prompt_setup_new.csv', index=False)